In [3]:
import wikipedia

articles = []
num_texts = 100
print(wikipedia.random(pages=10))
for title in wikipedia.random(pages=num_texts):
    try:
        page = wikipedia.page(title)
        articles.append(page.content)
    except wikipedia.exceptions.DisambiguationError as e:
        print(e.options)
    except wikipedia.exceptions.PageError as e:
        print(e)
        
print("articles: ", articles)


file_name = 'wiki_output.txt'
with open(file_name, 'wt') as f:
    for article in articles:
        f.write(article + '\n')

['Caldwell Municipal Airport (Kansas)', 'Colfax Township, Champaign County, Illinois', 'Punrasar Balaji', 'Billy Beer (footballer)', 'Cultural depictions of Eva Perón', 'Manino', 'Oncagnostus', 'Ossia', 'Andropogon', 'The Big Money']


c:\Users\Karyna\Miniconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Karyna\Miniconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


['Cárcel la 21', 'Tuol Sleng Genocide Museum']
Page id "albert crow" does not match any pages. Try another id!
['Big Brother 16 (disambiguation)', 'USS\xa0New Jersey\xa0(BB-16)']
['Aleksandr Yermakov', 'Aleksandr Ermakov (handballer)']
['Martin Johnson (rugby union)', 'Martin and Osa Johnson', 'Martin Johnson (musician)', 'Martin Johnson (racing driver)', 'Martin Hume Johnson', 'Martin Michael Johnson', 'Martin N. Johnson', 'Martin W. Johnson', 'Marty P. Johnson', 'Martin Johnson (Hollyoaks)', 'Martin Johnson (MP)', 'Martin Johnson (writer)', 'Martin Johnson (footballer)', 'Martin Johnston', 'Marty Johnstone']
articles:  ['Constitución is a small town in the Salto Department of northwestern Uruguay.\nConstitución is also the name of the municipality to which the town belongs.\n\n\n== Geography ==\nThe town is located 12 kilometres (7.5 mi) west of kilometre 535 of Route 3, about 38 kilometres (24 mi) north of the city of Salto, and on the east bank of the Uruguay River.\n\n\n== History

In [2]:
#1 Tokenize the whole text
file_name = 'wiki_output.txt'

import sentencepiece as spm
spm.SentencePieceTrainer.train(input=file_name, model_prefix='wikimodel', vocab_size=7200)

sp=spm.SentencePieceProcessor()
sp.load("wikimodel.model")
with open('wiki_output.txt', 'r') as file:
    # Read the contents of the file into a variable
    file_contents = file.read()
tokens=sp.encode_as_ids(file_contents)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: wiki_output.txt
  input_format: 
  model_prefix: wikimodel
  model_type: UNIGRAM
  vocab_size: 7200
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise_lev

In [3]:
print(tokens)

[1146, 3286, 182, 6380, 5294, 218, 1889, 4104, 5612, 30, 16, 13, 2687, 5003, 488, 4, 7117, 45, 4, 4094, 4, 9, 71, 8, 3, 950, 7, 8, 568, 6071, 5, 37, 285, 4033, 26, 6, 3583, 3193, 9, 3147, 5610, 4, 55, 133, 172, 12, 6, 333, 3976, 10, 3, 3600, 89, 8, 47, 6, 62, 246, 5, 2491, 890, 173, 4, 55, 4372, 12, 4961, 420, 9, 232, 54, 4272, 4, 53, 20, 5163, 12, 22, 3, 62, 2139, 7, 73, 1220, 14, 426, 10, 604, 14, 5384, 93, 4255, 5, 11, 4243, 7, 11, 5290, 5610, 1030, 119, 54, 3193, 64, 7109, 17, 1002, 691, 3002, 17, 6279, 74, 119, 2774, 33, 3490, 4854, 24, 4505, 7, 4, 17, 27, 6922, 2933, 37, 4488, 201, 1002, 691, 2229, 1733, 1521, 4761, 274, 6, 77, 4425, 5, 6, 0, 4937, 5592, 2572, 23, 3805, 3240, 1155, 4034, 5157, 30, 6, 77, 4423, 9, 203, 5187, 17, 6, 3805, 5307, 34, 3, 4505, 36, 2761, 6, 77, 5219, 2256, 7, 27, 308, 46, 181, 1229, 95, 6551, 4, 4485, 4, 3271, 18, 1504, 3580, 3599, 2761, 2879, 15, 276, 5, 11, 329, 6, 60, 11, 2761, 119, 3711, 337, 4991, 41, 3761, 95, 3185, 33, 11, 61, 7, 11, 11, 1300, 1

In [4]:
#CBOW Word2Vec model
#Firstly, we need to split the data into examples, e.g. (word, context word) in SKipGram and (context words, word) in CBOW
window=2
pairs=[]
for token_id in tokens[window:-window]:
    token_id=int(token_id)
    input_tokens=tokens[token_id-window:token_id]+tokens[token_id+1:token_id+window+1]
    output_token=tokens[token_id]
    pair=[input_tokens, output_token]
    pairs.append(pair)
    
print(pairs)


[[[4, 3271, 1504, 3580], 18], [[120, 3, 707, 8], 2293], [[760, 21, 407, 8], 1269], [[4, 1219, 74, 659], 1616], [[2982, 56, 813, 4], 1966], [[260, 496, 2209, 22], 3246], [[2883, 19, 714, 8], 248], [[568, 6071, 37, 285], 5], [[488, 4, 45, 4], 7117], [[13, 2687, 488, 4], 5003], [[255, 2850, 3, 641], 18], [[2861, 4, 4, 731], 3863], [[259, 5, 285, 5296], 37], [[182, 6380, 218, 1889], 5294], [[14, 4, 4, 68], 34], [[133, 172, 6, 333], 12], [[182, 6380, 218, 1889], 5294], [[659, 3389, 22, 179], 1611], [[182, 6380, 218, 1889], 5294], [[4104, 5612, 16, 13], 30], [[232, 54, 4, 53], 4272], [[1889, 4104, 30, 16], 5612], [[3286, 182, 5294, 218], 6380], [[2961, 5, 294, 886], 17], [[218, 1889, 5612, 30], 4104], [[1889, 4104, 30, 16], 5612], [[1646, 7, 92, 234], 5], [[5750, 43, 37, 6733], 6782], [[6380, 5294, 1889, 4104], 218], [[6, 3583, 9, 3147], 3193], [[19, 3566, 0, 2039], 6], [[151, 351, 1880, 3], 46], [[3, 950, 8, 568], 7], [[5294, 218, 4104, 5612], 1889], [[260, 147, 82, 7011], 3582], [[292, 522

In [11]:
# Hyperparameters
vocab_size = sp.get_piece_size()
embedding_dim = 128
window_size = 2
learning_rate = 0.01
epochs = 5

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, context_words):
        embedded = self.embeddings(context_words)
        projection = torch.mean(embedded, dim=1)
        out = self.linear(projection)
        log_probs = nn.functional.log_softmax(out, dim=1)
        return log_probs
    
    def embed(self, word):
        self.eval()
        embedding = self.embeddings(word)
        return embedding

In [14]:


# Convert pairs to torch tensors
torched_pairs=[]

for pair in pairs:
    context, target=pair
    torched_pairs.append([torch.tensor(context, dtype=torch.long), torch.tensor(target, dtype=torch.long)])

# cbow_dataset = DataLoader(torched_pairs, batch_size=50, shuffle=True)
    
cbow_model = CBOWModel(vocab_size, embedding_dim)  # Assuming CBOWModel is defined as before
loss_function = nn.NLLLoss()
optimizer = optim.SGD(cbow_model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    total_loss = 0
    for context, target in torched_pairs:
        # Forward pass
        cbow_model.zero_grad()
        log_probs = cbow_model(context)
        loss = loss_function(log_probs, target)

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x4 and 128x7200)